In [1]:
!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platfo

In [2]:
!pip install -q nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [4]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HuggingfaceAPI")

login(token=hf_token)


In [5]:
!pip install -q gensim

# Download fastText Urdu embeddings by Facebook
!wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ur.300.vec.gz
!gunzip cc.ur.300.vec.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 39.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.2.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Your loaded text files
with open("/kaggle/input/parallel-corpus-for-english-urdu-language/Dataset/english-corpus.txt", "r", encoding="utf-8") as f:
    english_lines = f.read().splitlines()

with open("/kaggle/input/parallel-corpus-for-english-urdu-language/Dataset/urdu-corpus.txt", "r", encoding="utf-8") as f:
    urdu_lines = f.read().splitlines()

# Filter misaligned lines
pairs = [(u, e) for u, e in zip(urdu_lines, english_lines) if u.strip() and e.strip()]
urdu_lines, english_lines = zip(*pairs)

df = pd.DataFrame({"translation": [{"ur": u, "en": e} for u, e in zip(urdu_lines, english_lines)]})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)


In [7]:
train_ds[0]

{'translation': {'en': 'i was wide awake', 'ur': 'میں وسیع جاگ رہا تھا'},
 '__index_level_0__': 11378}

In [8]:
import pandas as pd

# Extract English and Urdu translations
data = [{'en': item['translation']['en'], 'ur': item['translation']['ur']} for item in train_ds]

# Create DataFrame
df = pd.DataFrame(data)

print(df.head())


                     en                    ur
0      i was wide awake  میں وسیع جاگ رہا تھا
1  she broke into tears             وہ رو پڑی
2       we walked a lot         ہم بہت چل پڑے
3   i am in charge here  میں یہاں انچارج ہوں۔
4   there is no mistake     کوئی غلطی نہیں ہے


In [9]:
df["src_tok"] = df["ur"].apply(word_tokenize)
df["tgt_tok"] = df["en"].apply(word_tokenize)

In [10]:
# Function to build a word-to-index vocabulary from tokenized sentences
def build_vocab(tokenized_texts, min_freq=1):
    freq = {}  # Dictionary to store word frequencies

    # Count frequency of each word across all tokenized sentences
    for sent in tokenized_texts:
        for word in sent:
            freq[word] = freq.get(word, 0) + 1

    # Initialize vocabulary with special tokens and their reserved indices
    vocab = {
        "<pad>": 0,  # Padding token
        "<sos>": 1,  # Start of sequence
        "<eos>": 2,  # End of sequence
        "<unk>": 3   # Unknown token (for out-of-vocabulary words)
    }

    idx = 4  # Starting index for actual words

    # Add words to vocabulary that meet or exceed the minimum frequency threshold
    for word, count in freq.items():
        if count >= min_freq:
            vocab[word] = idx
            idx += 1

    return vocab  # Return the final vocabulary mapping

# Build vocabulary for source (Urdu) and target (English) tokenized texts
src_vocab = build_vocab(df["src_tok"])  # Urdu word-to-index mapping
tgt_vocab = build_vocab(df["tgt_tok"])  # English word-to-index mapping

# Create reverse vocabulary for target (index-to-word), useful for decoding predictions
inv_tgt_vocab = {i: w for w, i in tgt_vocab.items()}


In [11]:
# Load GloVe
def load_glove_embeddings(glove_path, vocab, embedding_dim=100):
    embeddings_index = {}
    with open(glove_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector

    matrix = np.zeros((len(vocab), embedding_dim))
    for word, i in vocab.items():
        vector = embeddings_index.get(word)
        if vector is not None:
            matrix[i] = vector
        else:
            matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))
    return torch.tensor(matrix, dtype=torch.float32)

# Download GloVe manually and update path
glove_path = "/kaggle/input/glove6b100dtxt/glove.6B.100d.txt"  # Ensure it's in working dir
embedding_dim = 100
tgt_embedding_matrix = load_glove_embeddings(glove_path, tgt_vocab, embedding_dim)

In [12]:
from torch.utils.data import Dataset, DataLoader
import torch

# Custom PyTorch dataset for translation tasks
class TranslationDataset(Dataset):
    def __init__(self, df, src_vocab, tgt_vocab, max_len=20):
        self.df = df                  # DataFrame containing tokenized source and target sentences
        self.src_vocab = src_vocab    # Vocabulary mapping for source language (e.g., Urdu)
        self.tgt_vocab = tgt_vocab    # Vocabulary mapping for target language (e.g., English)
        self.max_len = max_len        # Maximum length for input/output sequences (for padding)

    def __len__(self):
        # Returns total number of sentence pairs
        return len(self.df)

    def encode(self, tokens, vocab, add_sos_eos=True):
        # Ensures that token list isn't nested accidentally
        if isinstance(tokens[0], list):
            tokens = tokens[0]

        # Convert tokens to their corresponding indices, using <unk> if word is out of vocab
        ids = [vocab.get(tok, vocab["<unk>"]) for tok in tokens]

        # Optionally add <sos> and <eos> tokens
        if add_sos_eos:
            ids = [vocab["<sos>"]] + ids + [vocab["<eos>"]]

        # Pad or trim sequence to exactly max_len
        return ids[:self.max_len] + [vocab["<pad>"]] * max(0, self.max_len - len(ids))

    def __getitem__(self, idx):
        # Get the source and target token lists for the given index
        src_tokens = self.df.iloc[idx]["src_tok"]
        tgt_tokens = self.df.iloc[idx]["tgt_tok"]

        # Handle case where token lists might be nested (e.g., [['word1', 'word2']])
        if isinstance(src_tokens[0], list):
            src_tokens = src_tokens[0]
        if isinstance(tgt_tokens[0], list):
            tgt_tokens = tgt_tokens[0]

        # Encode token lists into padded sequences of token IDs
        src = self.encode(src_tokens, self.src_vocab)
        tgt = self.encode(tgt_tokens, self.tgt_vocab)

        # Return as PyTorch tensors
        return torch.tensor(src), torch.tensor(tgt)

# Instantiate the dataset with your DataFrame and vocabularies
train_dataset = TranslationDataset(df, src_vocab, tgt_vocab)

# Create a DataLoader to batch and shuffle data during training
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [14]:
import torch
import torch.nn as nn
import math

# Positional encoding module (as described in "Attention Is All You Need")
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout=0.1, maxlen=5000):
        super().__init__()

        # Compute the positional encodings once in log space
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)

        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)  # Even indices: sine
        pos_embedding[:, 1::2] = torch.cos(pos * den)  # Odd indices: cosine

        self.dropout = nn.Dropout(dropout)
        # Save positional encodings as a buffer so they're not trainable
        self.register_buffer('pos_embedding', pos_embedding.unsqueeze(1))

    def forward(self, token_embedding):
        # Add positional encodings to token embeddings and apply dropout
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


# Transformer-based seq2seq model with pretrained target embeddings
class ImprovedTransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_size, tgt_emb_matrix,
                 nhead=4, num_layers=4, dim_ff=512, dropout=0.1):
        super().__init__()
        self.emb_size = emb_size

        # Source token embedding (learnable)
        self.src_embedding = nn.Embedding(src_vocab_size, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size, dropout)

        # Target token embedding initialized from GloVe (trainable)
        self.tgt_embedding = nn.Embedding.from_pretrained(tgt_emb_matrix, freeze=False)
        self.pos_decoder = PositionalEncoding(emb_size, dropout)

        # PyTorch's built-in Transformer module
        self.transformer = nn.Transformer(
            d_model=emb_size,               # Embedding size (must match throughout)
            nhead=nhead,                    # Number of attention heads
            num_encoder_layers=num_layers, # Layers in encoder
            num_decoder_layers=num_layers, # Layers in decoder
            dim_feedforward=dim_ff,        # Size of feedforward sublayer
            dropout=dropout,               # Dropout rate
            batch_first=True               # Use batch-first tensor shape: (batch, seq, dim)
        )

        # Output layer: maps final decoder output to target vocabulary logits
        self.fc_out = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        # Apply source embedding + positional encoding
        src_emb = self.pos_encoder(self.src_embedding(src))

        # Apply target embedding (GloVe) + positional encoding
        tgt_emb = self.pos_decoder(self.tgt_embedding(tgt))

        # Pass through transformer encoder-decoder
        out = self.transformer(
            src_emb, tgt_emb,
            src_mask=src_mask, tgt_mask=tgt_mask,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask
        )

        # Project transformer output to vocabulary logits
        return self.fc_out(out)


In [15]:
model = ImprovedTransformerModel(
    src_vocab_size=len(src_vocab),
    tgt_vocab_size=len(tgt_vocab),
    emb_size=embedding_dim,
    tgt_emb_matrix=tgt_embedding_matrix,
    nhead=4,
    num_layers=4,
    dim_ff=512,
    dropout=0.1
)


In [16]:
from tqdm import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)

for epoch in range(10):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for src, tgt in loop:
        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])
        loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Update tqdm description
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} | Total Loss: {total_loss:.4f}")


Epoch 1: 100%|██████████| 11036/11036 [10:58<00:00, 16.77it/s, loss=5.29]


Epoch 1 | Total Loss: 61016.7812


Epoch 2: 100%|██████████| 11036/11036 [11:26<00:00, 16.08it/s, loss=4.44]


Epoch 2 | Total Loss: 59837.4122


Epoch 3: 100%|██████████| 11036/11036 [11:47<00:00, 15.61it/s, loss=5.02]


...

Epoch 8 | Total Loss: 59654.5586


Epoch 9: 100%|██████████| 11036/11036 [15:02<00:00, 12.23it/s, loss=5.03]


Epoch 9 | Total Loss: 59672.2464


Epoch 10: 100%|██████████| 11036/11036 [15:08<00:00, 12.15it/s, loss=5.29]

Epoch 10 | Total Loss: 59689.4144
